In [15]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
from datetime import datetime, timedelta

In [55]:
def read_file(filename):
    df_raw = pd.read_csv(filename)
    return df_raw

def preprocess(df_raw):
    df_stock = df_raw.filter(['Date', 'Close'], axis=1)
    df_stock.columns = ['ds', 'y']
    return df_stock
    
def train_model(df_stock):
    model = Prophet()
    model.fit(df_stock)
    return model

def predict(model, period):
    future_data = model.make_future_dataframe(periods = period)
    forecast = model.predict(future_data)
    return forecast

def get_forecasted_data(forecast, period):
    
    current_date = datetime.now().date()
    current_date = current_date.strftime('%d-%m-%Y')
    forecasted_till = datetime.now().date() + timedelta(days=period)
    forecasted_till = forecasted_till.strftime('%d-%m-%Y')
    forecast_range = forecast[(forecast.ds > current_date) & (forecast.ds <= forecasted_till)][["ds", "yhat","yhat_lower", "yhat_upper"]]
    
    return forecast_range

In [48]:
filename = 'data/data/data/BSE_BOM500002.csv'
df_raw = read_file(filename)
df_stock = preprocess(df_raw)

In [44]:
model = train_model(df_stock)

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [49]:

forecast = predict(model, 1000)
forecast.head()
#model.plot(forecast)

,ds,trend,trend_lower,trend_upper,yhat_lower,yhat_upper,seasonal,seasonal_lower,seasonal_upper,seasonalities,seasonalities_lower,seasonalities_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,yhat
0,1992-03-02,626.642719,626.642719,626.642719,348.039327,946.972972,29.680736,29.680736,29.680736,29.680736,29.680736,29.680736,13.021559,13.021559,13.021559,16.659177,16.659177,16.659177,656.323455
1,1992-03-03,626.671210,626.671210,626.671210,361.639064,946.794105,28.470053,28.470053,28.470053,28.470053,28.470053,28.470053,12.712019,12.712019,12.712019,15.758034,15.758034,15.758034,655.141263
2,1992-03-10,626.870644,626.870644,626.870644,361.604720,941.849867,23.207759,23.207759,23.207759,23.207759,23.207759,23.207759,12.712019,12.712019,12.712019,10.495739,10.495739,10.495739,650.078402
3,1992-03-11,626.899134,626.899134,626.899134,356.784715,976.190074,25.664504,25.664504,25.664504,25.664504,25.664504,25.664504,15.527831,15.527831,15.527831,10.136673,10.136673,10.136673,652.563638
4,1992-03-12,626.927625,626.927625,626.927625,339.144396,949.116243,21.627224,21.627224,21.627224,21.627224,21.627224,21.627224,11.708874,11.708874,11.708874,9.918350,9.918350,9.918350,648.554849


In [58]:
forecast_range = get_forecasted_data(forecast, 1000)
forecast_range.shape

(989, 4)

In [93]:
import os
#def trainAll(foldername):
models = list()
for filename in os.listdir('dataTest'):
    print filename    
    if os.path.exists(filename):
        df_raw = read_file(filename)
        df_stock = preprocess(df_raw)
        model = train_model(df_stock)
        print model
        models.append(model)

BSE_BOM500003.csv
BSE_BOM500002.csv
BSE_BOM500002_0.csv
BSE_BOM500008.csv
BSE_BOM500009.csv


In [81]:
def predictAll(models):
    forecasts = list()
    for i in range(0, len(models)):
        forecast = predict(models[i], 1000)
        forecasts.append(forecast)
    return forecasts
        #forecast_range = get_forecasted_data(forecast, 1000)
        #forecast_range.shape
        #return forecast_range

In [82]:
models = trainAll('dataTest')
print len(models)
forecasts = predictAll(models)

0


In [78]:
print len(forecasts)

0


In [105]:
import os
import pandas as pd
import datetime

codes = pd.read_csv('BSE-datasets-codes.csv', sep=',', header = None)
#print codes
df1 = codes.to_dict(orient = 'records')
#print df1

dict_codes = dict()
for element in df1:
    dict_codes[element[0]] = element[1]
    
#print dict_codes

for filename in os.listdir('output/prediction/mprophet'):

    df = pd.read_csv('output/prediction/mprophet/' + filename, sep='\t')
    df = df.filter(['ds', 'yhat'], axis=1)
    df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%d")
    now = datetime.datetime.now()
    
    todays_price = str(df.loc[df['ds'] == str(now.date()), 'yhat']).split()[1]
    print todays_price
    
    mask = (df['ds'] > str(now.date()))
    
    df = df.loc[mask]
    max_price = str(df.loc[df['yhat'].idxmax()]).split()[4]
    max_price_date = str(df.loc[df['yhat'].idxmax()]).split()[1]
    print max_price
    last_row = df.tail(1)
    predicted_price = str(last_row['yhat']).split()[1]
    print predicted_price
    
    file_code = filename.split('.')[0].split('_')
    #print file_code
    
    
    company_name = dict_codes[file_code[0] + '/' + file_code[1]] 
    #print company_name
    
    data = {'name': company_name, 'today_price': todays_price , 'predicted_price': predicted_price, 'max_price': max_price, 'max_price_date': max_price_date}
data_modelp['company_name'] = data
    #json_data = json.dumps(data)
    
    


1509.755733
1513.58
1484.982909
1509.755733
1513.58
1484.982909


NameError: name 'data_modelp' is not defined